In [2]:
using DataFrames
using CSV
using Printf
import Base.Iterators: flatten, zip
import IterTools.subsets

# Define struct

In [3]:
struct Rule
    ant::Array{Int64,1}
    con::Array{Int64,1}
end

# Data load

In [4]:
onigiri_data = CSV.read("onigiri.csv", delim=",")
onigiri_array = Matrix(onigiri_data)

5×5 Array{Int64,2}:
 0  1  1  1  0
 1  1  0  0  0
 1  0  0  1  1
 0  1  1  1  0
 0  1  0  1  0

In [5]:
function GTI(mat::Array{Int64,2}, indexes::Array{Int64,1})
    return mat[:,indexes] .==1
end

GTI (generic function with 1 method)

In [6]:
function support(array_2d::Array{Int64,2}, indexes::Array{Int64,1}; m="num")
    gti_b = GTI(array_2d, indexes)
    if size(gti_b)[2] ==0
        return 0
    end
    b = all(gti_b, dims=2)
    if m =="num"
        return sum(b)
    elseif m == "ratio"
        return sum(b)/length(b)
    elseif m=="bool"
        return b
    end
end

support (generic function with 1 method)

In [55]:
function support2(array_2d::Array{Int64,2}, indexes::Array{Int64,1})
    gti_b = GTI(array_2d, indexes)
    if size(gti_b)[2] ==0
        return 0
    end
    b = all(gti_b, dims=2)
    return b
end

support2 (generic function with 1 method)

In [7]:
function confidence(array_2d::Array{Int64,2}, 
        X_indexes::Array{Int64,1}, Y_indexes::Array{Int64,1})
    sup_X = support(array_2d, X_indexes)
    X_Y_indexes = cat(X_indexes, Y_indexes, dims=1)
    return support(array_2d, X_Y_indexes)/sup_X
end

confidence (generic function with 1 method)

In [8]:
function getF1(array_2D::Array{Int64,2}, minsup::Float64)
    return [[col] for col in 1:size(array_2D)[2] if support(array_2D, [col], m="ratio") >= minsup]
end

getF1 (generic function with 1 method)

In [9]:
function getFkPlusOne(array_2D::Array{Int64,2}, indexes::Array{Array{Int64,1},1}, minsup::Float64)
    return [col for col in indexes if support(array_2D, col, m="ratio") >= minsup]
                
end

getFkPlusOne (generic function with 1 method)

In [10]:
function getCkPlusOne(prevCandidate::Array{Array{Int64, 1}, 1}, k)
    @assert all(length.(prevCandidate) .==  k-1)
    @assert k>1
    items = unique(collect(flatten(prevCandidate)))
    tmp_candidates = [x for x in subsets(items, k)]
    if k ==2
        return tmp_candidates
    end
    
    candidates = [
        candidate for candidate in tmp_candidates
        if all(
                x in prevCandidate
                for x in subsets(candidate, k-1))
    ]
                
    return candidates
                
end

getCkPlusOne (generic function with 1 method)

In [11]:
function isEmpty(F::Array{Array{Int64,1},1})
    if length(F) < 1
        return true
    else
        return false
    end
end

isEmpty (generic function with 1 method)

In [12]:
function isCalcConfNeeded(array_prev_ant::Array{Array{Int64,1},1},
                    array_ant::Array{Array{Int64,1},1}, set_f::Array{Int64,1})
    array_prev_con = [setdiff(set_f,  set_c) for set_c in array_ant]
    array_con = [setdiff(set_f, set_c) for set_c in array_ant]
    
    out = []
    for (a,c) in zip(array_ant, array_con)
        out_inner = []
        for i in 1:length(c)

            array_ant_candidate = a
            cand = c[i]
            array_candidate_ant = vcat(a, cand)
            array_candidate_con = filter(x ->x != cand, c)
            
            res = any([issubset(array_candidate_ant, i) for i in array_prev_ant])
            append!(out_inner, res)
        end
        if all(out_inner)
            append!(out, true)
        else
            append!(out, false)
        end
    end
    
    out = convert(Array{Bool, 1}, out)

    return out
end

                
            

isCalcConfNeeded (generic function with 1 method)

In [13]:
function frequent(array_2D::Array{Int64,2}; minsup::Float64)
    k = 1
    F_now = getF1(array_2D, minsup)
    F_list = []
    F_table = zeros(1,size(array_2D)[2]) # first line is dummy (all zero)
    @printf "k=1 len of items is %d"  length(F_now)
    println()
    append!(F_list, [F_now])
    
    
    
    while(true)
        C_next = getCkPlusOne(F_now, k+1)
        F_next = getFkPlusOne(array_2D, C_next, minsup)
        
        if isEmpty(F_next)
            break
        end
        k += 1
        F_now = F_next
        append!(F_list, [F_now])
        @printf "k=%d len of items is %d"  k length(F_now)
        println()
    end
    
    F_list = convert(Array{Array{Array{Int64,1},1},1}, F_list)
    
    return F_list
end

frequent (generic function with 1 method)

In [14]:
_F_list = frequent(onigiri_array, minsup=0.4)

k=1 len of items is 4
k=2 len of items is 3
k=3 len of items is 1


3-element Array{Array{Array{Int64,1},1},1}:
 [[1], [2], [3], [4]]    
 [[2, 3], [2, 4], [3, 4]]
 [[2, 3, 4]]             

In [15]:
function itemlist2table(item_list::Array{Array{Array{Int64,1},1},1},
                                             col_num::Int64)
    table = []
    for k in item_list
        for item in k
            #label encode
            arr = zeros(Int64, 1,col_num)
            arr[1,item] .= 1
            
            table = cat(table, arr, dims=1)
        end
    end
    return table
end

            

itemlist2table (generic function with 1 method)

In [16]:
tmp = itemlist2table( _F_list, 10)

8×10 Array{Any,2}:
 1  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0
 0  1  1  0  0  0  0  0  0  0
 0  1  0  1  0  0  0  0  0  0
 0  0  1  1  0  0  0  0  0  0
 0  1  1  1  0  0  0  0  0  0

In [17]:
function find_rules(array_2D::Array{Int64, 2}, 
        F_list::Array{Array{Array{Int64,1},1},1}; minconf::Float64)
    conf_list = []
    
    
    for F in F_list
        k = length(F[1])
        
        if k == 1
            #conf_list = vcat(conf_list, Any{[Rule([0],[0])]}) # DUMMY!!
        
        elseif k == 2
            conf_list_k = []
            for f_2 in F
                A = f_2[1]
                B = f_2[2]
                conf_AB = confidence(array_2D, [A], [B])
                if conf_AB >= minconf
                    #append!(conf_list_k, Rule([A],[B]))
                    conf_list_k = vcat(conf_list_k, Rule([A],[B]))

                end
                conf_BA = confidence(array_2D, [B], [A])
                if conf_BA >= minconf
                    #append!(conf_list_k, Rule([B],[A]))
                    conf_list_k = vcat(conf_list_k, Rule([B],[A]))
                end
            end
            append!(conf_list, [conf_list_k])   


        elseif k >= 3
            conf_list_k = []
            for f_k in F
                
                j = 1
                
                array_antecedent =  collect(subsets(f_k, k-1))
                array_consequent = [setdiff(f_k,  set_c) for set_c in array_antecedent]
                conf = [confidence(array_2D, ant, con) for (ant, con) in zip(array_antecedent, array_consequent)]
                isHigher = conf .>= minconf
                if sum(isHigher) > 0
                    array_antecedent_filtered_by_conf = array_antecedent[isHigher]
                    array_consequent_filtered_by_conf = array_consequent[isHigher]
                    append!(conf_list_k, [Rule(a,c) for (a,c) in zip(array_antecedent_filtered_by_conf,
                                                                                            array_consequent_filtered_by_conf)])
                    
                    while(j < k-1)
                        array_antecedent_new = collect(subsets(f_k, k-(j+1)))
                        _res = isCalcConfNeeded(array_antecedent_filtered_by_conf, array_antecedent_new, f_k)
                        if sum(_res) > 0
                            array_antecedent_filtered_by_prev = array_antecedent_new[_res]
                            array_consequent_filtered_by_prev = [setdiff(f_k,  set_c) 
                                                                                            for set_c in array_antecedent_filtered_by_prev]
                            conf = [confidence(array_2D, ant, con) for (ant, con) in zip(array_antecedent_filtered_by_prev, 
                                                                                                                                array_consequent_filtered_by_prev)]
                            isHigher = conf .>= minconf
                            if sum(isHigher) > 0
                                array_antecedent_filtered_by_prev_and_conf = array_antecedent_filtered_by_prev[isHigher]
                                array_consequent_filtered_by_prev_and_conf = array_consequent_filtered_by_prev[isHigher]
                                append!(conf_list_k, [Rule(a,c) for (a,c) in zip(array_antecedent_filtered_by_prev_and_conf, 
                                                                                                         array_consequent_filtered_by_prev_and_conf)])
                            end
                        end
                        j += 1
                    end #while
                end
            end
            append!(conf_list, [conf_list_k])
        end
    end
    conf_list = convert(Array{Array{Rule,1},1}, conf_list)
    return conf_list
end

find_rules (generic function with 1 method)

In [18]:
find_rules(onigiri_array, _F_list, minconf=0.7)

2-element Array{Array{Rule,1},1}:
 [Rule([3], [2]), Rule([2], [4]), Rule([4], [2]), Rule([3], [4])]
 [Rule([2, 3], [4]), Rule([3, 4], [2]), Rule([3], [2, 4])]       

## Load data (store data)

In [ ]:
store_data = CSV.read("store_data_trans.csv", delim=",")
store_array = Matrix(store_data)
store_array

In [ ]:
minsup = 0.003
F_list = frequent(store_array, minsup=minsup)

In [ ]:
minconf = 0.01
find_rules(store_array, F_list, minconf)

# mnist

In [19]:
mnist_data = CSV.read("mnist_8x8_image.csv", delim=",",  header=false)
mnist_array = Matrix(mnist_data)
mnist_array = convert(Array{Int64, 2}, mnist_array)

mnist_label = CSV.read("mnist_8x8_label.csv", delim=",",  header=false)
mnist_label = Matrix(mnist_label)
mnist_label = convert(Array{Int64, 2}, mnist_label)

mnist_label_onehot = CSV.read("mnist_8x8_label_onehot.csv", delim=",",  header=false)
mnist_label_onehot = Matrix(mnist_label_onehot)
mnist_label_onehot = convert(Array{Int64, 2}, mnist_label_onehot)


898×10 Array{Int64,2}:
 1  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0
 ⋮              ⋮            
 0  0  0  0  1  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0
 0  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  0
 0  0  1  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  1  0  0  0  0
 0  0  0  0  1  0  0  0  0  0

In [20]:
SubArray(mnist_array, mnist_label .==3)

MethodError: MethodError: no method matching SubArray(::Array{Int64,2}, ::BitArray{2})
Closest candidates are:
  SubArray(::AbstractArray, !Matched::Tuple) at subarray.jl:21
  SubArray(!Matched::IndexCartesian, ::P, !Matched::I, !Matched::Tuple{Vararg{Any,N}}) where {P, I, N} at subarray.jl:25
  SubArray(!Matched::IndexLinear, ::P, !Matched::I, !Matched::Tuple{Vararg{Any,N}}) where {P, I, N} at subarray.jl:29

## concat image and label

In [21]:
mnist_cat = cat(mnist_array, mnist_label, dims=2)

898×65 Array{Int64,2}:
 0  0  0  1  1  0  0  0  0  0  1  1  1  …  1  0  0  0  0  0  1  1  0  0  0  0
 0  0  0  1  1  0  0  0  0  0  0  1  1     0  0  0  0  0  0  1  1  1  0  0  1
 0  0  0  0  1  1  0  0  0  0  0  1  1     1  0  0  0  0  0  0  1  1  1  0  2
 0  0  0  1  1  0  0  0  0  1  1  0  1     1  1  0  0  0  0  1  1  1  0  0  3
 0  0  0  0  1  0  0  0  0  0  0  0  1     1  0  0  0  0  0  0  1  0  0  0  4
 0  0  1  1  0  0  0  0  0  0  1  1  1  …  1  0  0  0  0  1  1  1  1  0  0  5
 0  0  0  1  1  0  0  0  0  0  0  1  1     1  1  0  0  0  0  1  1  1  0  0  6
 0  0  0  1  1  1  1  0  0  0  0  0  0     0  0  0  0  0  1  0  0  0  0  0  7
 0  0  1  1  1  0  0  0  0  0  1  1  1     1  1  0  0  0  1  1  1  1  0  0  8
 0  0  1  1  0  0  0  0  0  0  1  1  1     1  0  0  0  0  1  1  1  0  0  0  9
 0  0  0  1  1  1  0  0  0  0  1  1  1  …  1  0  0  0  0  0  1  1  0  0  0  0
 0  0  0  0  1  1  0  0  0  0  0  0  1     1  0  0  0  0  0  0  1  1  0  0  1
 0  0  0  1  0  0  0  0  0  0  1  1  0   

# Julia

In [ ]:
mnist_data

In [ ]:
@time F_list = frequent(mnist_array, minsup=0.15)

In [ ]:
F_list[3]

In [ ]:
F_table = itemlist2table(F_list, size(mnist_cat)[2])

In [ ]:
size(mnist_cat)[2]

In [ ]:
@time F_list = frequent(mnist_cat, minsum=0.15)

In [ ]:
@time F_list = frequent(mnist_cat, minsum=0.09)

In [ ]:
@time F_list = frequent(mnist_label, minsum=0.09)

In [ ]:
@time find_rules(mnist_array, F_list, minconf=0.1)

In [ ]:
mnist_array

In [22]:
a = support(mnist_array, [10,11,12], m="bool")
b = support(mnist_array, [10,11], m="bool")
c = support(mnist_array, [12], m="bool")

898×1 BitArray{2}:
  true
  true
  true
 false
 false
  true
  true
 false
  true
  true
  true
 false
  true
     ⋮
  true
  true
  true
  true
  true
 false
 false
  true
  true
  true
  true
  true

In [23]:
@time [support(mnist_array, [10,11,12,13,14,15,16,17],m="bool") for i in 1:100000]

  6.836749 seconds (2.92 M allocations: 5.958 GiB, 17.20% gc time)


100000-element Array{BitArray{2},1}:
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 ⋮                               
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]

In [24]:
d = support(mnist_array,[10,11,12,13,14,15,16],m="bool")
@time [all(hcat(d,
    support(mnist_array, [17], m="bool")), dims=2)
    for i in 1:100000]

  2.614109 seconds (3.39 M allocations: 1.288 GiB, 13.35% gc time)


100000-element Array{BitArray{2},1}:
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 ⋮                               
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]
 [false; false; … ; false; false]

In [81]:
_colnum = size(mnist_array)[2] # 64
tmp = collect(subsets(1:_colnum, 4))

635376-element Array{Array{Int64,1},1}:
 [1, 2, 3, 4]    
 [1, 2, 3, 5]    
 [1, 2, 3, 6]    
 [1, 2, 3, 7]    
 [1, 2, 3, 8]    
 [1, 2, 3, 9]    
 [1, 2, 3, 10]   
 [1, 2, 3, 11]   
 [1, 2, 3, 12]   
 [1, 2, 3, 13]   
 [1, 2, 3, 14]   
 [1, 2, 3, 15]   
 [1, 2, 3, 16]   
 ⋮               
 [59, 60, 62, 63]
 [59, 60, 62, 64]
 [59, 60, 63, 64]
 [59, 61, 62, 63]
 [59, 61, 62, 64]
 [59, 61, 63, 64]
 [59, 62, 63, 64]
 [60, 61, 62, 63]
 [60, 61, 62, 64]
 [60, 61, 63, 64]
 [60, 62, 63, 64]
 [61, 62, 63, 64]

In [70]:
length(tmp)  * 4 * 64 

162656256

In [80]:
function GTI(mat::Array{Int64,2}, indexes::Array{Int64,1})
    return mat[:,indexes] .==1
end

GTI (generic function with 2 methods)

In [79]:
@time [GTI(mnist_array, t) for t in tmp]

@time GTI.(Ref(mnist_array), tmp)

110.918201 seconds (4.50 M allocations: 19.984 GiB, 90.18% gc time)


InterruptException: InterruptException:

In [53]:
function f(x,y)
    return x+y
end

x = 1
yy = 1:100000000

@time [f(x,y) for y in yy]

@time f.(Ref(x), yy)

 10.759150 seconds (200.08 M allocations: 3.729 GiB, 41.81% gc time)
  0.909368 seconds (9 allocations: 762.940 MiB, 0.42% gc time)


100000000-element Array{Int64,1}:
         2
         3
         4
         5
         6
         7
         8
         9
        10
        11
        12
        13
        14
         ⋮
  99999990
  99999991
  99999992
  99999993
  99999994
  99999995
  99999996
  99999997
  99999998
  99999999
 100000000
 100000001